In [3]:
import pandas as pd
import re
import phonenumbers
from phonenumbers import carrier
from phonenumbers.phonenumberutil import number_type
 
 # The read_csv is reading the csv file into Dataframe
pd.options.display.max_columns = 47
pd.options.display.max_rows = 1000

#df = pd.read_csv("dirty.csv", low_memory=False )
phoneFrame = pd.read_csv('dirty.csv', usecols=['contact_phone'])


def phone_validation(phoneNumber):
    try:
        number2Validate = phoneNumber
        return carrier._is_mobile(number_type(phonenumbers.parse(number2Validate)))
    except phonenumbers.NumberParseException:
        return False

# Применение функции валидации к столбцу 'contact_phone'
phoneFrame['phone_is_valid'] = phoneFrame['contact_phone'].apply(phone_validation)

# Очищаем значения в 'contact_phone', если номер не мобильный
phoneFrame.loc[~phoneFrame['phone_is_valid'], 'contact_phone'] = None  # или '' для пустой строки

# Сохранение DataFrame в текстовый файл
phoneFrame.to_csv('phone_output.txt', sep='\t', index=False)  # Используйте sep=',' для запятой

print("Данные сохранены в файл phone_output.txt")

emailFrame = pd.read_csv('dirty.csv', usecols=['contact_email'])
 
# Функция для проверки валидности адреса электронной почты
def email_validation(email):
    if isinstance(email, str):  # Проверяем, является ли email строкой
        # Регулярное выражение для проверки формата электронной почты
        email_regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        return re.match(email_regex, email) is not None
    return False  # Если email не строка, возвращаем False

# Применение функции проверки к столбцу 'contact_email'
emailFrame['email_is_valid'] = emailFrame['contact_email'].apply(email_validation)

# Очищаем значения в 'contact_email', если адрес не валидный
emailFrame.loc[~emailFrame['email_is_valid'], 'contact_email'] = None  # или '' для пустой строки

# Сохранение DataFrame в текстовый файл
emailFrame.to_csv('output_emails.txt', sep='\t', index=False)  # Используйте sep=',' для запятой

print("Данные сохранены в файл output_emails.txt")

zipFrame = pd.read_csv('dirty.csv', usecols=['addr_zip'])

# Функция для проверки валидности почтового индекса в России
def zip_validation(zip_code):
    if isinstance(zip_code, str):  # Проверяем, является ли zip_code строкой
        zip_regex = r'^\d{6}$'  # Формат: 123456
        return re.match(zip_regex, zip_code) is not None
    return False  # Если zip_code не строка, возвращаем False

# Применение функции проверки к столбцу 'addr_zip'
zipFrame['zip_is_valid'] = zipFrame['addr_zip'].apply(zip_validation)

# Очищаем значения в 'addr_zip', если индекс не валидный
zipFrame.loc[~zipFrame['zip_is_valid'], 'addr_zip'] = None  # или '' для пустой строки

# Сохранение DataFrame в текстовый файл
zipFrame.to_csv('output_zips.txt', sep='\t', index=False)  # Используйте sep=',' для запятой

print("Данные сохранены в файл output_zips.txt")

Данные сохранены в файл phone_output.txt
Данные сохранены в файл output_emails.txt


C:\Users\evoxed\AppData\Local\Temp\ipykernel_27584\4156772543.py:54: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  zipFrame = pd.read_csv('dirty.csv', usecols=['addr_zip'])


Данные сохранены в файл output_zips.txt
